### Resources Used

- https://docs.rapids.ai/api/cudf/stable/10min.html
- https://www.dataquest.io/blog/data-science-portfolio-machine-learning/
- https://docs.dask.org/en/latest/dataframe-best-practices.html
- https://docs.dask.org/en/latest/setup/single-distributed.html#localcluster
- Dataset: https://docs.rapids.ai/datasets/mortgage-data
- https://dask-cuda.readthedocs.io/en/latest/specializations.html
- #!conda create -n rapids-0.16 -c rapidsai -c nvidia -c conda-forge -c defaults rapids=0.16 python=3.7 cudatoolkit=11.0 matplotlib=3.3.3 gcsfs=0.7.1


## Check Environment

In [ ]:
%%bash
nvidia-smi
nvcc --version

In [1]:
import numpy as np; print('numpy Version:', np.__version__)
import pandas as pd; print('pandas Version:', pd.__version__)
import xgboost as xgb; print('XGBoost Version:', xgb.__version__)
import cudf; print('cudf Version:', cudf.__version__)
import cuml; print('cudf Version:', cuml.__version__)
import gcsfs; print('gcsfs Version:', gcsfs.__version__)
import time
import dask_cudf; print('dask_cudf Version:', gcsfs.__version__)
import dask; print('dask Version:', gcsfs.__version__)
import dask.dataframe as dask_df
import glob;
import matplotlib; print('matplotlib Version:', matplotlib.__version__)
from dask.diagnostics import ProgressBar
from dask.distributed import Client, progress, wait


numpy Version: 1.19.4
pandas Version: 1.1.5
XGBoost Version: 1.3.0-SNAPSHOT
cudf Version: 0.16.0
cudf Version: 0.16.0
gcsfs Version: 0.7.1
dask_cudf Version: 0.7.1
dask Version: 0.7.1
matplotlib Version: 3.3.2


## Data Types

In [2]:
col_acq = ['LoanID','Channel','SellerName','OrInterestRate','OrUnpaidPrinc','OrLoanTerm',
        'OrDate','FirstPayment','OrLTV','OrCLTV','NumBorrow','DTIRat','CreditScore',
        'FTHomeBuyer','LoanPurpose','PropertyType','NumUnits','OccStatus','PropertyState',
        'Zip','MortInsPerc','ProductType','CoCreditScore','MortInsType','RelMortInd']

col_per = ['LoanID','MonthRep','Servicer','CurrInterestRate','CAUPB','LoanAge','MonthsToMaturity',
          'AdMonthsToMaturity','MaturityDate','MSA','CLDS','ModFlag','ZeroBalCode','ZeroBalDate',
          'LastInstallDate','ForeclosureDate','DispositionDate','PPRC','AssetRecCost','MHRC',
          'ATFHP','NetSaleProceeds','CreditEnhProceeds','RPMWP','OFP','NIBUPB','PFUPB','RMWPF',
          'FPWA','ServicingIndicator']

parse_dates_acq =['OrDate','FirstPayment']
parse_dates_per =['MonthRep','MaturityDate','ZeroBalDate','LastInstallDate','ForeclosureDate','DispositionDate']

dtype_acq={ "LoanID":"int","Channel":"str","SellerName":"str","OrInterestRate":"float","OrUnpaidPrinc":"float","OrLoanTerm":"float","OrDate":"str",
   "FirstPayment":"str","OrLTV":"float","OrCLTV":"float",  "NumBorrow":"float", "DTIRat":"float", "CreditScore":"float", "FTHomeBuyer":"str",
   "LoanPurpose":"str", "PropertyType":"str", "NumUnits":"float", "OccStatus":"str",  "PropertyState":"str",  "Zip":"int", "MortInsPerc":"float",
   "ProductType":"str", "CoCreditScore":"float", "MortInsType":"float", "RelMortInd":"str"}

dtype_per={"LoanID":"int","MonthRep":"str","Servicer":"str", "CurrInterestRate":"float", "CAUPB":"float", "LoanAge":"float","MonthsToMaturity":"float",
   "AdMonthsToMaturity":"float", "MaturityDate":"str", "MSA":"float", "CLDS":"float", "ModFlag":"str", "ZeroBalCode":"float", "ZeroBalDate":"str",
    "LastInstallDate":"str",  "ForeclosureDate":"str", "DispositionDate":"str", "PPRC":"float", "AssetRecCost":"float", "MHRC":"float", "ATFHP":"float",
    "NetSaleProceeds":"float", "CreditEnhProceeds":"float","RPMWP":"float","OFP":"float","NIBUPB":"float", "PFUPB":"float", "RMWPF":"float",
   "FPWA":"str", "ServicingIndicator":"str"
}


## Test on small sample

In [3]:
# sample_acq_fnames='gs://mchrestkha-github-ml-examples/fannie_mae_loans/acq/Acquisition_2016Q1.txt'
# sample_perf_fnames='gs://mchrestkha-github-ml-examples/fannie_mae_loans/perf/Performance_2016Q1.txt'

# df_acq = pd.read_csv(sample_acq_fnames, sep='|', names=col_acq, index_col=False, dtype=dtype_acq, parse_dates=parse_dates_acq)
# df_per = pd.read_csv(sample_perf_fnames, sep='|', names=col_per, index_col=False, dtype=dtype_per, parse_dates=parse_dates_per)

## Run on full population

### Dask_cuDF

In [4]:
from dask_cuda import LocalCUDACluster
from dask.distributed import Client
# Create a Dask Cluster with one worker per GPU
num_workers=2
processes=True
cluster = LocalCUDACluster(n_workers=num_workers, processes=processes)
# cluster = LocalCUDACluster()
client = Client(cluster)
client

Client Scheduler: tcp://127.0.0.1:43775 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 2 Cores: 2 Memory: 179.38 GB


In [5]:
acq_fnames='gs://mchrestkha-github-ml-examples/fannie_mae_loans/acq/Acquisition_201*'
perf_fnames='gs://mchrestkha-github-ml-examples/fannie_mae_loans/perf/Performance_201*'
!gsutil du -sh 'gs://mchrestkha-github-ml-examples/fannie_mae_loans/acq/'
!gsutil du -sh 'gs://mchrestkha-github-ml-examples/fannie_mae_loans/perf/'

/snap/google-cloud-sdk/162/lib/third_party/requests/__init__.py:83: RequestsDependencyWarning: Old version of cryptography ([1, 2, 3]) may cause slowdown.
  warnings.warn(warning, RequestsDependencyWarning)
4.01 GiB     gs://mchrestkha-github-ml-examples/fannie_mae_loans/acq
/snap/google-cloud-sdk/162/lib/third_party/requests/__init__.py:83: RequestsDependencyWarning: Old version of cryptography ([1, 2, 3]) may cause slowdown.
  warnings.warn(warning, RequestsDependencyWarning)
190.96 GiB   gs://mchrestkha-github-ml-examples/fannie_mae_loans/perf


In [6]:
#%time df_acq = dask_cudf.read_csv(acq_fnames, sep='|', names=col_acq, dtype=dtype_acq, parse_dates=parse_dates_acq)
%time df_per = dask_cudf.read_csv(perf_fnames, sep='|', names=col_per, dtype=dtype_per, parse_dates=parse_dates_per)

CPU times: user 1.15 s, sys: 661 ms, total: 1.81 s
Wall time: 4.11 s


In [ ]:
print(type(df_per))
print(df_per.npartitions)
print(df_per.shape)

In [ ]:
rows=df_per.shape[0].persist() # start computation in the background
progress(rows)      # watch progress (takes ~3 min)

In [ ]:
rows=rows.compute()
cols=df_per.shape[1]
print(rows,',', cols) 

In [ ]:
df_per_1loan=df_per.drop_duplicates(subset='LoanID', keep='last', inplace=False).persist()
progress(df_per_1loan)      # watch progress (takes ~3 min)

In [ ]:
del df_per_1loan
del rows
# del df_per_desc

In [7]:
df_per_desc=df_per.describe().persist()
progress(df_per_desc)      # watch progress 

VBox()

In [8]:
df_per_desc=df_per_desc.compute()
df_per_desc

,ATFHP,AdMonthsToMaturity,AssetRecCost,CAUPB,CLDS,CreditEnhProceeds,CurrInterestRate,LoanAge,LoanID,MHRC,MSA,MonthsToMaturity,NIBUPB,NetSaleProceeds,OFP,PFUPB,PPRC,RMWPF,RPMWP,ZeroBalCode
count,11404.000000,5.683560e+08,9884.000000,4.853730e+08,5.689803e+08,13006.000000,5.689803e+08,5.689803e+08,5.689803e+08,5901.000000,5.689803e+08,5.689803e+08,6924.000000,10722.000000,448.000000,624314.000000,13263.000000,329.000000,3450.000000,5.468040e+06
mean,1253.420575,2.639612e+02,7169.169378,1.898636e+05,-1.331150e-03,126106.352694,4.019165e+00,2.629323e+01,4.750422e+01,1915.180191,2.758539e+04,2.725358e+02,5367.822400,4813.221921,83046.175643,1348.223943,5001.977686,10178.437254,44136.579741,1.030975e+00
std,2888.108139,9.623637e+01,8358.342868,1.142066e+05,4.714148e-01,92911.662259,6.430933e-01,1.982052e+01,NaN,1839.677489,1.391756e+04,9.056833e+01,25376.995677,5949.186262,87317.360591,8639.325233,4447.388388,16802.095683,50615.718478,4.692321e-01
min,-16901.599609,0.000000e+00,5.000000,0.000000e+00,-2.000000e+00,0.000000,1.750000e+00,-2.000000e+00,-2.147483e+09,0.000000,0.000000e+00,-1.000000e+00,0.000000,0.000000,0.000000,0.000000,0.600000,0.000000,0.000000,1.000000e+00
25%,297.910004,2.830000e+02,3188.434937,1.295044e+05,0.000000e+00,81175.927734,3.750000e+00,1.300000e+01,-1.002994e+09,1094.352509,1.782000e+04,2.960000e+02,356.035004,1916.119965,42278.132812,0.000000,3534.849976,0.000000,25076.794922,1.000000e+00
50%,1166.715027,3.340000e+02,5411.087402,2.045942e+05,0.000000e+00,132757.625000,4.375000e+00,2.900000e+01,8.344316e+07,1851.905029,3.110000e+04,3.370000e+02,1234.119995,3676.180176,67659.992188,0.000000,5213.122559,0.000000,39710.777344,1.000000e+00
75%,7691.000000,3.550000e+02,14549.403320,7.918387e+05,0.000000e+00,238664.765625,5.250000e+00,5.300000e+01,1.223547e+09,3830.000000,4.014000e+04,3.550000e+02,4503.345093,8769.165039,137807.601562,94666.671875,9328.490234,13529.689453,68713.562500,1.500000e+01
max,80564.023438,4.800000e+02,85684.234375,1.403000e+06,9.100000e+01,942587.375000,8.500000e+00,1.310000e+02,2.147484e+09,34452.000000,4.974000e+04,4.830000e+02,450003.687500,102833.742188,473630.250000,238809.250000,142121.171875,52994.332031,597175.687500,1.600000e+01


In [ ]:
print(type(df_per_1loan))
print(df_per_1loan.npartitions)
print(df_per_1loan.shape)

In [ ]:
#df_per_1loan=df_per_1loan.compute() 
print(type(df_per_1loan))
#print(df_per_1loan.npartitions)
print(df_per_1loan.shape)

In [ ]:
#df_per_1loan.hist(column='LoanAge')
loan_age=df_per_1loan[['LoanAge','LoanID']].groupby('LoanAge',as_index=False).count()
loan_age.plot.line()

In [ ]:
print(type(out))
y=out.compute()
print(type(y))
y.head()

In [ ]:
y= x.compute()
type(df_per)

In [ ]:
type(test)

In [ ]:
rows=df_acq.shape[0].compute()
cols=df_acq.shape[1]
print(rows,',', cols) 
rows=df_per.shape[0].compute()
cols=df_per.shape[1]
print(rows,',', cols) 

In [ ]:
#df_per.head()
df_per.iloc[:,1:5].describe()
df_per.describe()

In [ ]:
df_acq['RelMortInd'].describe()
df_acq['RelMortInd'].unique()
df_per['CLDS'].describe()


In [ ]:
#%time print('Describe:',df_per.describe())
#%time print('Describe:',df_per.isna().any())


In [ ]:
df_per_1loan=df_per.drop_duplicates(subset='LoanID', keep='last', inplace=False)
print(df_per.shape)
print(df_per_1loan.shape)
df_per_1loan.head()